# Data quality analysis for `recommended_metadata`

This table holds extra metadata relating the product and the merchant. Let's start, as usual, analysing it's schema.

In [0]:
import pyspark.sql.functions as F
import plotly.express as px
import plotly.graph_objects as go

In [0]:
%sql
DESCRIBE mvp.bronze_feeds.recommended_metadata

col_name,data_type,comment
aw_deep_link,string,The unique Awin tracking link for the product.
product_name,string,The full title or name of the product.
aw_product_id,bigint,Awin's unique internal identifier for the product.
merchant_product_id,string,The unique ID assigned to the product by the merchant.
merchant_image_url,string,Direct URL to the product image on the merchant's server.
description,string,Detailed description of the product features.
merchant_category,string,The category name assigned by the merchant.
search_price,double,The current selling price used for search filtering.
merchant_name,string,The name of the advertiser/store.
merchant_id,bigint,The unique Awin identifier for the merchant.


Fields with null description are the same from `default_product_info`, using the same approach as in `product_specification`, let's just keep the relevant identiers between them, that means `aw_product_id` and `merchant_product_id`.

From the perspective of data type, the schema looks almost ok, price and cost are double, except by display_price, identifiers are bigint, remaining values as categories and liks are strings. it would be useful if `last_updated` was a timestamp fields, but we may try to convert it if the field has information.

Let's start by checking the distribution of null values over the table, then we will decide which fields could be useful or not.

In [0]:
columns_to_keep = [
    "aw_product_id",
    "merchant_product_id",

    "data_feed_id",
    "merchant_id",
    "category_id",
    
    "merchant_name",
    
    "category_name",
    
    "aw_image_url",
    "currency",
    "merchant_deep_link",
    "language",
    "last_updated",
    
    "store_price",
    "delivery_cost",
    "display_price",
]

bronze_recommended_metadata = \
    spark.read.table("mvp.bronze_feeds.recommended_metadata")\
      .select(*columns_to_keep)
display(bronze_recommended_metadata.limit(10))

aw_product_id,merchant_product_id,data_feed_id,merchant_id,category_id,merchant_name,category_name,aw_image_url,currency,merchant_deep_link,language,last_updated,store_price,delivery_cost,display_price
42205909314,275877880,109288,118977,null,Lauri Esporte,null,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-38-688148508c37f-large.jpg&feedId=109288&k=6ac3d79ed4153442e642760111e741ce06acc173,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,null,null,null,null,BRL1399.00
42205908872,275877881,109288,118977,null,Lauri Esporte,null,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-39-6884501a489f1-large.jpg&feedId=109288&k=cbbf94cb41effb077dff90454ad989d0e04b68b3,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,null,null,null,null,BRL1399.00
42205909434,275877882,109288,118977,null,Lauri Esporte,null,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-40-6884501c88e8b-large.jpg&feedId=109288&k=428aeb199ef0bbb1718ed7c432f41298784652cc,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,null,null,null,null,BRL1399.00
42205909548,275877883,109288,118977,null,Lauri Esporte,null,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-41-6884501d260d2-large.jpg&feedId=109288&k=fdd50a1999693684d53c985decc9fa028c84f1fd,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,null,null,null,null,BRL1399.00
42205907626,275877884,109288,118977,null,Lauri Esporte,null,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-42-6884501d3d6a9-large.jpg&feedId=109288&k=32235830ced416fcb6dc9b9098f8a42a8ca74695,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,null,null,null,null,BRL1399.00
42205907978,275877885,109288,118977,null,Lauri Esporte,null,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-43-6884501d3b8b5-large.jpg&feedId=109288&k=365c43124b8137099a53d9257cef11a86c75e435,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,null,null,null,null,BRL1399.00
42205909326,275877886,109288,118977,null,Lauri Esporte,null,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-44-6884501d39665-large.jpg&feedId=109288&k=5ac7226ba7a8469712053aa13e349622e1519a29,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,null,null,null,null,BRL1399.00
42205907629,275878129,109288,118977,null,Lauri Esporte,null,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-38-68800dcd430eb-large.jpg&feedId=109288&k=23db7ac141da3c05a3dc933b545fe2e056be14b0,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudpulse-masculino-bege/p,null,null,null,null,BRL1099.00
42205908914,275878131,109288,118977,null,Lauri Esporte,null,https://images2.p

In [0]:
cols = bronze_recommended_metadata.columns

stats = []
df = bronze_recommended_metadata

total = df.count()
for col in cols:
    null_count = df.filter(F.col(col).isNull()).count()
    null_pct = (null_count / total) * 100 if total > 0 else 0
    stats.append({
        "column": col,
        "null_count": null_count,
        "total": total,
        "null_pct": null_pct
    })

import pandas as pd

stats_df = pd.DataFrame(stats)

fig = go.Figure()

fig.add_bar(
    x=stats_df["column"],
    y=stats_df["null_pct"],
    text=[f"{col}<br>{pct:.1f}%" for col, pct in zip(stats_df["column"], stats_df["null_pct"])],
    textposition="outside",
    name="Null Percentage"
)

fig.update_layout(
    title="Columns null percentage",
    xaxis_title="Column",
    yaxis_title="Null Percentage (%)",
    showlegend=False,
    height=700,
    width=1100,
    margin=dict(l=120, r=40, t=80, b=180),
    xaxis=dict(
        tickmode='array',
        tickvals=list(stats_df["column"]),
        ticktext=list(stats_df["column"]),
        tickangle=45
    )
)

fig.update_traces(marker_line_width=1.5)

display(fig)

We can see that some fields have a lot of nulls, `category_id`, `category_name`, `language`, `last_updated`, `store_price` and `delivery_cost`, let's just remove them.

In [0]:
columns_to_keep = [
    "aw_product_id",
    "merchant_product_id",

    "data_feed_id",
    "merchant_id",
    
    "merchant_name",
        
    "aw_image_url",
    "currency",
    "merchant_deep_link",

    "display_price",
]

bronze_recommended_metadata = bronze_recommended_metadata.select(*columns_to_keep)
display(bronze_recommended_metadata.limit(10))

aw_product_id,merchant_product_id,data_feed_id,merchant_id,merchant_name,aw_image_url,currency,merchant_deep_link,display_price
42205909314,275877880,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-38-688148508c37f-large.jpg&feedId=109288&k=6ac3d79ed4153442e642760111e741ce06acc173,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00
42205908872,275877881,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-39-6884501a489f1-large.jpg&feedId=109288&k=cbbf94cb41effb077dff90454ad989d0e04b68b3,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00
42205909434,275877882,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-40-6884501c88e8b-large.jpg&feedId=109288&k=428aeb199ef0bbb1718ed7c432f41298784652cc,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00
42205909548,275877883,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-41-6884501d260d2-large.jpg&feedId=109288&k=fdd50a1999693684d53c985decc9fa028c84f1fd,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00
42205907626,275877884,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-42-6884501d3d6a9-large.jpg&feedId=109288&k=32235830ced416fcb6dc9b9098f8a42a8ca74695,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00
42205907978,275877885,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-43-6884501d3b8b5-large.jpg&feedId=109288&k=365c43124b8137099a53d9257cef11a86c75e435,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00
42205909326,275877886,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-44-6884501d39665-large.jpg&feedId=109288&k=5ac7226ba7a8469712053aa13e349622e1519a29,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00
42205907629,275878129,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-38-68800dcd430eb-large.jpg&feedId=109288&k=23db7ac141da3c05a3dc933b545fe2e056be14b0,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudpulse-masculino-bege/p,BRL1099.00
42205908914,275878131,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-39-68800dcf9b0e5-large.jpg&feedId=109288&k=6a9668837e01d5153273061b7ffe23f02edb43ca,BRL,https://www.lauriesporte.com.br/tenis-on-running-c

The field `display_price` is of type string, but as a price, we would expect a double. Let's change it by removing the prefix `BRL` then casting to double, what is not possible to cast, still has incorrect format and we can decide later what to do.

In [0]:
bronze_recommended_metadata_temp = bronze_recommended_metadata\
    .withColumnRenamed("display_price", "display_price_string")\
    .withColumn("display_price", F.replace(F.col("display_price_string"), F.lit("BRL"), F.lit("")).try_cast("double"))
display(bronze_recommended_metadata_temp.limit(10))

aw_product_id,merchant_product_id,data_feed_id,merchant_id,merchant_name,aw_image_url,currency,merchant_deep_link,display_price_string,display_price
42205909314,275877880,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-38-688148508c37f-large.jpg&feedId=109288&k=6ac3d79ed4153442e642760111e741ce06acc173,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00,1399.0
42205908872,275877881,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-39-6884501a489f1-large.jpg&feedId=109288&k=cbbf94cb41effb077dff90454ad989d0e04b68b3,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00,1399.0
42205909434,275877882,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-40-6884501c88e8b-large.jpg&feedId=109288&k=428aeb199ef0bbb1718ed7c432f41298784652cc,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00,1399.0
42205909548,275877883,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-41-6884501d260d2-large.jpg&feedId=109288&k=fdd50a1999693684d53c985decc9fa028c84f1fd,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00,1399.0
42205907626,275877884,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-42-6884501d3d6a9-large.jpg&feedId=109288&k=32235830ced416fcb6dc9b9098f8a42a8ca74695,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00,1399.0
42205907978,275877885,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-43-6884501d3b8b5-large.jpg&feedId=109288&k=365c43124b8137099a53d9257cef11a86c75e435,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00,1399.0
42205909326,275877886,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-44-6884501d39665-large.jpg&feedId=109288&k=5ac7226ba7a8469712053aa13e349622e1519a29,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,BRL1399.00,1399.0
42205907629,275878129,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-38-68800dcd430eb-large.jpg&feedId=109288&k=23db7ac141da3c05a3dc933b545fe2e056be14b0,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudpulse-masculino-bege/p,BRL1099.00,1099.0
42205908914,275878131,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-39-68800dcf9b0e5-large.jpg&feedId=109288&k=6a9668837e01d51532

Let's check where the value was not null but became null after the cast (where the error still exists):

In [0]:
display(bronze_recommended_metadata_temp.where(F.col("display_price").isNull() & F.col("display_price_string").isNotNull()))
display(bronze_recommended_metadata_temp.where(F.col("display_price").isNull() & F.col("display_price_string").isNull()))

aw_product_id,merchant_product_id,data_feed_id,merchant_id,merchant_name,aw_image_url,currency,merchant_deep_link,display_price_string,display_price
42405292474,2420270,null,null,"36 23,5 a 24,3",Vestuário e acessórios > Roupas,99.9,null,BRL,null
42405290723,2420271,null,null,"36 23,5 a 24,3",Vestuário e acessórios > Roupas,99.9,null,BRL,null
42405290724,2420272,null,null,"36 23,5 a 24,3",Vestuário e acessórios > Roupas,99.9,null,BRL,null
42405292610,2420274,null,null,"36 23,5 a 24,3",Vestuário e acessórios > Roupas,99.9,null,BRL,null


aw_product_id,merchant_product_id,data_feed_id,merchant_id,merchant_name,aw_image_url,currency,merchant_deep_link,display_price_string,display_price


There is just a few rows (just 4), and these four rows have a straing merchant name, what means the rows are malformed, not only the fields. Also, there is no row where `display_price` was null before the cast, we can safely remove these rows (all rows where `search_price` is null after casting, as they are just malformed)

In [0]:
columns_to_keep = [
    "aw_product_id",
    "merchant_product_id",

    "data_feed_id",
    "merchant_id",
    
    "merchant_name",
        
    "aw_image_url",
    "currency",
    "merchant_deep_link",

    "display_price",
]

bronze_recommended_metadata = bronze_recommended_metadata_temp.where(F.col("display_price").isNotNull()).select(*columns_to_keep)
display(bronze_recommended_metadata.limit(10))

aw_product_id,merchant_product_id,data_feed_id,merchant_id,merchant_name,aw_image_url,currency,merchant_deep_link,display_price
42205909314,275877880,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-38-688148508c37f-large.jpg&feedId=109288&k=6ac3d79ed4153442e642760111e741ce06acc173,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205908872,275877881,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-39-6884501a489f1-large.jpg&feedId=109288&k=cbbf94cb41effb077dff90454ad989d0e04b68b3,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205909434,275877882,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-40-6884501c88e8b-large.jpg&feedId=109288&k=428aeb199ef0bbb1718ed7c432f41298784652cc,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205909548,275877883,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-41-6884501d260d2-large.jpg&feedId=109288&k=fdd50a1999693684d53c985decc9fa028c84f1fd,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205907626,275877884,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-42-6884501d3d6a9-large.jpg&feedId=109288&k=32235830ced416fcb6dc9b9098f8a42a8ca74695,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205907978,275877885,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-43-6884501d3b8b5-large.jpg&feedId=109288&k=365c43124b8137099a53d9257cef11a86c75e435,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205909326,275877886,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-44-6884501d39665-large.jpg&feedId=109288&k=5ac7226ba7a8469712053aa13e349622e1519a29,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205907629,275878129,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-38-68800dcd430eb-large.jpg&feedId=109288&k=23db7ac141da3c05a3dc933b545fe2e056be14b0,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudpulse-masculino-bege/p,1099.0
42205908914,275878131,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-39-68800dcf9b0e5-large.jpg&feedId=109288&k=6a9668837e01d5153273061b7ffe23f02edb43ca,BRL,https://www.lauriesporte.com.br/tenis-on-running-cloudpulse-masculino-bege/p,1099.

Now, let's check the range of values for display_price.

In [0]:
# Calculate absolute min and max
min_val = bronze_recommended_metadata.agg(F.min("display_price")).first()[0]
max_val = bronze_recommended_metadata.agg(F.max("display_price")).first()[0]

# Calculate quartiles using approxQuantile
q1, median, q3 = bronze_recommended_metadata.approxQuantile("display_price", [0.25, 0.5, 0.75], 0.01)

# Count records in each quartile
quartile_counts = [
    bronze_recommended_metadata.filter(F.col("display_price") <= q1).count(),
    bronze_recommended_metadata.filter((F.col("display_price") > q1) & (F.col("display_price") <= median)).count(),
    bronze_recommended_metadata.filter((F.col("display_price") > median) & (F.col("display_price") <= q3)).count(),
    bronze_recommended_metadata.filter(F.col("display_price") > q3).count()
]

# Create summary table
summary_data = [
    ("Minimum", min_val, None),
    ("Q1 (25%)", q1, quartile_counts[0]),
    ("Median (50%)", median, quartile_counts[1]),
    ("Q3 (75%)", q3, quartile_counts[2]),
    ("Maximum", max_val, quartile_counts[3])
]

summary_df = spark.createDataFrame(summary_data, ["Statistic", "Value", "Record_Count"])
display(summary_df)

Statistic,Value,Record_Count
Minimum,1.0,null
Q1 (25%),79.99,291756
Median (50%),132.9,246879
Q3 (75%),229.99,276471
Maximum,8495.0,277880


The behavior for `display_price` looks similar for that we found in `default_product_info`. I understand, for now, there is no need for futher analysis. It's clear that values are a bit different, what may come from the randomization used in `approxQuantile` function, but this difference is not relevant.

Let's check the different values for the other categorical columns, that is `merchant_name`, `merchant_id` and `currency`, the columns containing URLs are not relevant. Given the description for `data_feed_id` it's just the identifier for a feed file, and we are not interested in it.

In [0]:
import plotly.express as px

merchant_counts = bronze_recommended_metadata.groupBy("merchant_id", "merchant_name")\
    .count()\
    .orderBy(F.desc("count"))\
    .limit(100)\
    .toPandas()

merchant_counts["merchant_id"] = merchant_counts["merchant_id"].astype(str)
merchant_counts["merchant_label"] = merchant_counts["merchant_id"] + " - " + merchant_counts["merchant_name"]

fig = px.bar(
    merchant_counts,
    x="merchant_label",
    y="count",
    title="Top 100 Merchant ID & Name Pairs by Row Count",
    log_y=True
)

fig.update_layout(
    xaxis_title="Merchant ID - Name",
    yaxis_title="Row Count (Log Scale)",
    xaxis_tickangle=45,
    height=700,
    width=1100,
    margin=dict(l=120, r=40, t=80, b=180)
)

display(fig)

Now let's check the `currency` column

In [0]:
top_currencies = bronze_recommended_metadata.groupBy("currency")\
    .count()\
    .orderBy(F.desc("count"))\
    .limit(100)\
    .toPandas()

fig = px.bar(
    top_currencies,
    x="currency",
    y="count",
    title="Top 100 Currencies by Row Count",
    log_y=True
)

fig.update_layout(
    xaxis_title="Currency",
    yaxis_title="Row Count (Log Scale)",
    xaxis_tickangle=45,
    height=700,
    width=1100,
    margin=dict(l=120, r=40, t=80, b=180)
)

display(fig)

The `currency` column has only one value (BRL) we can just remove it as it's redundant. We are also not interested in the 

On the end, the transformations that bring us from bronze to silver for this table is.

In [0]:
columns_to_keep = [
    "aw_product_id",
    "merchant_product_id",

    "data_feed_id",
    "merchant_id",
    
    "merchant_name",
        
    "aw_image_url",
    "merchant_deep_link",

    "display_price",
]

bronze_recommended_metadata = \
    spark.read.table("mvp.bronze_feeds.recommended_metadata")\
      .withColumn("display_price", F.replace(F.col("display_price"), F.lit("BRL"), F.lit("")).try_cast("double"))\
      .where(F.col("display_price").isNotNull())\
      .select(*columns_to_keep)

display(bronze_recommended_metadata.limit(10))

aw_product_id,merchant_product_id,data_feed_id,merchant_id,merchant_name,aw_image_url,merchant_deep_link,display_price
42205909314,275877880,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-38-688148508c37f-large.jpg&feedId=109288&k=6ac3d79ed4153442e642760111e741ce06acc173,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205908872,275877881,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-39-6884501a489f1-large.jpg&feedId=109288&k=cbbf94cb41effb077dff90454ad989d0e04b68b3,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205909434,275877882,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-40-6884501c88e8b-large.jpg&feedId=109288&k=428aeb199ef0bbb1718ed7c432f41298784652cc,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205909548,275877883,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-41-6884501d260d2-large.jpg&feedId=109288&k=fdd50a1999693684d53c985decc9fa028c84f1fd,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205907626,275877884,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-42-6884501d3d6a9-large.jpg&feedId=109288&k=32235830ced416fcb6dc9b9098f8a42a8ca74695,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205907978,275877885,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-43-6884501d3b8b5-large.jpg&feedId=109288&k=365c43124b8137099a53d9257cef11a86c75e435,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205909326,275877886,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudmonster-2-masculino-cinza-44-6884501d39665-large.jpg&feedId=109288&k=5ac7226ba7a8469712053aa13e349622e1519a29,https://www.lauriesporte.com.br/tenis-on-running-cloudmonster-2-masculino-cinza-2/p,1399.0
42205907629,275878129,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-38-68800dcd430eb-large.jpg&feedId=109288&k=23db7ac141da3c05a3dc933b545fe2e056be14b0,https://www.lauriesporte.com.br/tenis-on-running-cloudpulse-masculino-bege/p,1099.0
42205908914,275878131,109288,118977,Lauri Esporte,https://images2.productserve.com/?w=200&h=200&bg=white&trim=5&t=letterbox&url=ssl%3Aimages.yampi.me%2Fassets%2Fstores%2Flauri-esporte%2Fuploads%2Fimages%2Ftenis-on-running-cloudpulse-masculino-bege-bege-39-68800dcf9b0e5-large.jpg&feedId=109288&k=6a9668837e01d5153273061b7ffe23f02edb43ca,https://www.lauriesporte.com.br/tenis-on-running-cloudpulse-masculino-bege/p,1099.0
42205908687,275878133,109288,118977,Lauri E